In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statistics
import random

# Feature Selection
# Filter
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.feature_selection import VarianceThreshold
from kydavra import MultiSURFSelector
# Wrapper
import shap
from sklearn.feature_selection import RFE
from boruta import BorutaPy

# model
import xgboost as xgb
from xgboost import XGBClassifier

# parameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# evaluate
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


from sklearn.feature_selection import SelectKBest

# from functools import reduce

In [2]:
# for threshold
def filter_arr(sample_arr, filter_index):
    result_val = sample_arr[filter_index]
    result_index = filter_index
    compare_index = filter_index + 1
    if sample_arr[0] == sample_arr[filter_index]:
        return (0,0)
        
    if (len(sample_arr) -1 == filter_index):
        compare_index = filter_index - 1
    
    if (sample_arr[filter_index] == sample_arr[compare_index]):
        for index, val in enumerate(sample_arr):
            if (val == result_val):
                result_val = sample_arr[index - 1]
                result_index = index - 1

    return (result_val, result_index)



In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv('LSVT_voice_rehabilitation_prepareddata.csv', header=0)
df

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,Jitter->F0_TKEO_prc25,Jitter->F0_TKEO_prc75,Jitter->F0_TKEO_prc95,Jitter->F0_FM,Jitter->F0range_5_95_perc,Jitter->pitch_abs,Jitter->pitch_percent,Jitter->pitch_PQ5_classical_Schoentgen,Jitter->pitch_PQ5_classical_Baken,Jitter->pitch_PQ5_generalised_Schoentgen,Jitter->pitch_abs0th_perturb,Jitter->pitch_CV,Jitter->pitch_TKEO_mean,Jitter->pitch_TKEO_std,Jitter->pitch_TKEO_prc5,Jitter->pitch_TKEO_prc25,Jitter->pitch_TKEO_prc75,Jitter->pitch_TKEO_prc95,Jitter->pitch_FM,Jitter->pitch_range_5_95_perc,Shimmer->Ampl_abs_dif,Shimmer->Ampl_dif_percent,Shimmer->Ampl_PQ3_classical_Schoentgen,Shimmer->Ampl_PQ3_classical_Baken,Shimmer->Ampl_PQ3_generalised_Schoentgen,Shimmer->Ampl_PQ5_classical_Schoentgen,Shimmer->Ampl_PQ5_classical_Baken,Shimmer->Ampl_PQ5_generalised_Schoentgen,Shimmer->Ampl_PQ11_classical_Schoentgen,Shimmer->Ampl_PQ11_classical_Baken,Shimmer->Ampl_PQ11_generalised_Schoentgen,Shimmer->Ampl_abs0th_perturb,Shimmer->Ampl_dB,Shimmer->Ampl_CV,Shimmer->Ampl_TKEO_mean,Shimmer->Ampl_TKEO_std,Shimmer->Ampl_TKEO_prc5,Shimmer->Ampl_TKEO_prc25,Shimmer->Ampl_TKEO_prc75,Shimmer->Ampl_TKEO_prc95,Shimmer->Ampl_AM,HNR->HNR_dB_Praat_mean,HNR->HNR_dB_Praat_std,NHR->NHR_Praat_mean,NHR->NHR_Praat_std,OQ->prc5_95,OQ->std_cycle_open,OQ->std_cycle_closed,RPDE,DFA,PPE,GNE->mean,GNE->std,GNE->SNR_TKEO,GNE->SNR_SEO,GNE->NSR_TKEO,GNE->NSR_SEO,VFER->mean,VFER->std,VFER->entropy,VFER->SNR_TKEO1,VFER->SNR_TKEO,VFER->SNR_SEO,VFER->NSR_TKEO1,VFER->NSR_TKEO,VFER->NSR_SEO,IMF->SNR_SEO,IMF->SNR_TKEO,IMF->SNR_entropy,IMF->NSR_SEO,IMF->NSR_TKEO,IMF->NSR_entropy,Log energy,MFCC_0th coef,MFCC_1st coef,MFCC_2nd coef,MFCC_3rd coef,MFCC_4th coef,MFCC_5th coef,MFCC_6th coef,MFCC_7th coef,MFCC_8th coef,MFCC_9th coef,MFCC_10th coef,MFCC_11th coef,MFCC_12th coef,delta log energy,0th delta,1st delta,2nd delta,3rd delta,4th delta,5th delta,6th delta,7th delta,8th delta,9th delta,10th delta,11th delta,12th delta,delta delta log energy,delta delta 0th,1st delta delta,2nd delta-delta,3rd delta-delta,4th delta-delta,5th delta-delta,6th delta-delta,7th delta-delta,8th delta-delta,9th delta-delta,10th delta-delta,11th delta-delta,12th delta-delta,F0_series_F0_expTitze,std_F0_series_F0_expTitze,prc95_5_F0_series_F0_expTitze,Data_length,Ea,Ed_1_coef,Ed_2_coef,Ed_3_coef,Ed_4_coef,Ed_5_coef,Ed_6_coef,Ed_7_coef,Ed_8_coef,Ed_9_coef,Ed_10_coef,entropy_shannon_1_coef,entropy_shannon_2_coef,entropy_shannon_3_coef,entropy_shannon_4_coef,entropy_shannon_5_coef,entropy_shannon_6_coef,entropy_shannon_7_coef,entropy_shannon_8_coef,entropy_shannon_9_coef,entropy_shannon_10_coef,entropy_log_1_coef,entropy_log_2_coef,entropy_log_3_coef,entropy_log_4_coef,entropy_log_5_coef,entropy_log_6_coef,entropy_log_7_coef,entropy_log_8_coef,entropy_log_9_coef,entropy_log_10_coef,det_TKEO_mean_1_coef,det_TKEO_mean_2_coef,det_TKEO_mean_3_coef,det_TKEO_mean_4_coef,det_TKEO_mean_5_coef,det_TKEO_mean_6_coef,det_TKEO_mean_7_coef,det_TKEO_mean_8_coef,det_TKEO_mean_9_coef,det_TKEO_mean_10_coef,det_TKEO_std_1_coef,det_TKEO_std_2_coef,det_TKEO_std_3_coef,det_TKEO_std_4_coef,det_TKEO_std_5_coef,det_TKEO_std_6_coef,det_TKEO_std_7_coef,det_TKEO_std_8_coef,det_TKEO_std_9_coef,det_TKEO_std_10_coef,entropy_shannon2_1_coef,entropy_shannon2_2_coef,entropy_shannon2_3_coef,entropy_shannon2_4_coef,entropy_shannon2_5_coef,entropy_shannon2_6_coef,entropy_shannon2_7_coef,entropy_shannon2_8_coef,entropy_shannon2_9_coef,entropy_shannon2_10_coef,entropy_log2_1_coef,entropy_log2_2_coef,entropy_log2_3_coef,entropy_log2_4_coef,entropy_log2_5_coef,entropy_log2_6_coef,entropy_log2_7_coef,entropy_log2_8_coef,entropy_log2_9_coef,entropy_log2_10_coef,det_TKEO_mean2_1_coef,det_TKEO_mean2_2_coef,det_TKEO_mean2_3_coef,det_TKEO_mean2_4_coef,det_TKEO_mean2_5_coef,det_TKEO_mean2_6_coef,det_TKEO_mean2_

In [5]:
col_names = list(df.columns[:-2])
features = df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df1 = pd.DataFrame(features, columns = col_names)
df1['Gender'] = df['Gender']
df1['class']  = df['class']

df1

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,Jitter->F0_TKEO_prc25,Jitter->F0_TKEO_prc75,Jitter->F0_TKEO_prc95,Jitter->F0_FM,Jitter->F0range_5_95_perc,Jitter->pitch_abs,Jitter->pitch_percent,Jitter->pitch_PQ5_classical_Schoentgen,Jitter->pitch_PQ5_classical_Baken,Jitter->pitch_PQ5_generalised_Schoentgen,Jitter->pitch_abs0th_perturb,Jitter->pitch_CV,Jitter->pitch_TKEO_mean,Jitter->pitch_TKEO_std,Jitter->pitch_TKEO_prc5,Jitter->pitch_TKEO_prc25,Jitter->pitch_TKEO_prc75,Jitter->pitch_TKEO_prc95,Jitter->pitch_FM,Jitter->pitch_range_5_95_perc,Shimmer->Ampl_abs_dif,Shimmer->Ampl_dif_percent,Shimmer->Ampl_PQ3_classical_Schoentgen,Shimmer->Ampl_PQ3_classical_Baken,Shimmer->Ampl_PQ3_generalised_Schoentgen,Shimmer->Ampl_PQ5_classical_Schoentgen,Shimmer->Ampl_PQ5_classical_Baken,Shimmer->Ampl_PQ5_generalised_Schoentgen,Shimmer->Ampl_PQ11_classical_Schoentgen,Shimmer->Ampl_PQ11_classical_Baken,Shimmer->Ampl_PQ11_generalised_Schoentgen,Shimmer->Ampl_abs0th_perturb,Shimmer->Ampl_dB,Shimmer->Ampl_CV,Shimmer->Ampl_TKEO_mean,Shimmer->Ampl_TKEO_std,Shimmer->Ampl_TKEO_prc5,Shimmer->Ampl_TKEO_prc25,Shimmer->Ampl_TKEO_prc75,Shimmer->Ampl_TKEO_prc95,Shimmer->Ampl_AM,HNR->HNR_dB_Praat_mean,HNR->HNR_dB_Praat_std,NHR->NHR_Praat_mean,NHR->NHR_Praat_std,OQ->prc5_95,OQ->std_cycle_open,OQ->std_cycle_closed,RPDE,DFA,PPE,GNE->mean,GNE->std,GNE->SNR_TKEO,GNE->SNR_SEO,GNE->NSR_TKEO,GNE->NSR_SEO,VFER->mean,VFER->std,VFER->entropy,VFER->SNR_TKEO1,VFER->SNR_TKEO,VFER->SNR_SEO,VFER->NSR_TKEO1,VFER->NSR_TKEO,VFER->NSR_SEO,IMF->SNR_SEO,IMF->SNR_TKEO,IMF->SNR_entropy,IMF->NSR_SEO,IMF->NSR_TKEO,IMF->NSR_entropy,Log energy,MFCC_0th coef,MFCC_1st coef,MFCC_2nd coef,MFCC_3rd coef,MFCC_4th coef,MFCC_5th coef,MFCC_6th coef,MFCC_7th coef,MFCC_8th coef,MFCC_9th coef,MFCC_10th coef,MFCC_11th coef,MFCC_12th coef,delta log energy,0th delta,1st delta,2nd delta,3rd delta,4th delta,5th delta,6th delta,7th delta,8th delta,9th delta,10th delta,11th delta,12th delta,delta delta log energy,delta delta 0th,1st delta delta,2nd delta-delta,3rd delta-delta,4th delta-delta,5th delta-delta,6th delta-delta,7th delta-delta,8th delta-delta,9th delta-delta,10th delta-delta,11th delta-delta,12th delta-delta,F0_series_F0_expTitze,std_F0_series_F0_expTitze,prc95_5_F0_series_F0_expTitze,Data_length,Ea,Ed_1_coef,Ed_2_coef,Ed_3_coef,Ed_4_coef,Ed_5_coef,Ed_6_coef,Ed_7_coef,Ed_8_coef,Ed_9_coef,Ed_10_coef,entropy_shannon_1_coef,entropy_shannon_2_coef,entropy_shannon_3_coef,entropy_shannon_4_coef,entropy_shannon_5_coef,entropy_shannon_6_coef,entropy_shannon_7_coef,entropy_shannon_8_coef,entropy_shannon_9_coef,entropy_shannon_10_coef,entropy_log_1_coef,entropy_log_2_coef,entropy_log_3_coef,entropy_log_4_coef,entropy_log_5_coef,entropy_log_6_coef,entropy_log_7_coef,entropy_log_8_coef,entropy_log_9_coef,entropy_log_10_coef,det_TKEO_mean_1_coef,det_TKEO_mean_2_coef,det_TKEO_mean_3_coef,det_TKEO_mean_4_coef,det_TKEO_mean_5_coef,det_TKEO_mean_6_coef,det_TKEO_mean_7_coef,det_TKEO_mean_8_coef,det_TKEO_mean_9_coef,det_TKEO_mean_10_coef,det_TKEO_std_1_coef,det_TKEO_std_2_coef,det_TKEO_std_3_coef,det_TKEO_std_4_coef,det_TKEO_std_5_coef,det_TKEO_std_6_coef,det_TKEO_std_7_coef,det_TKEO_std_8_coef,det_TKEO_std_9_coef,det_TKEO_std_10_coef,entropy_shannon2_1_coef,entropy_shannon2_2_coef,entropy_shannon2_3_coef,entropy_shannon2_4_coef,entropy_shannon2_5_coef,entropy_shannon2_6_coef,entropy_shannon2_7_coef,entropy_shannon2_8_coef,entropy_shannon2_9_coef,entropy_shannon2_10_coef,entropy_log2_1_coef,entropy_log2_2_coef,entropy_log2_3_coef,entropy_log2_4_coef,entropy_log2_5_coef,entropy_log2_6_coef,entropy_log2_7_coef,entropy_log2_8_coef,entropy_log2_9_coef,entropy_log2_10_coef,det_TKEO_mean2_1_coef,det_TKEO_mean2_2_coef,det_TKEO_mean2_3_coef,det_TKEO_mean2_4_coef,det_TKEO_mean2_5_coef,det_TKEO_mean2_6_coef,det_TKEO_mean2_

In [6]:
# df1['class'] = df1.apply(lambda row: categorise(row), axis=1)
print('class 0 = ',df1['class'].value_counts()[0],'ตัว')
print('class 1 = ',df1['class'].value_counts()[1],'ตัว')
print('Majority Class= ', df1['class'].value_counts()[0]/len(df1))

class 0 =  42 ตัว
class 1 =  84 ตัว
Majority Class=  0.3333333333333333


In [7]:
# threshold:

log_2base = math.log(len(df1),2.0)
base2n = int(round(log_2base,0))
p2_5 = int(round(len(df1.columns)*0.025,0))
p5 =  int(round(len(df1.columns)*0.05,0))
p10 =  int(round(len(df1.columns)*0.1,0))
p20 =  int(round(len(df1.columns)*0.2,0))

print('treshold logbase2(n):', base2n)
print('treshold 2.5%:',p2_5)
print('treshold 5%:',p5)
print('treshold 10%:',p10)
print('treshold 20%:',p20)

thres_d = {'base2n':base2n,
              'p2_5':p2_5,
              'p5':p5,
              'p10':p10,
              'p20':p20}


thres_d


treshold logbase2(n): 7
treshold 2.5%: 8
treshold 5%: 16
treshold 10%: 31
treshold 20%: 63


{'base2n': 7, 'p2_5': 8, 'p5': 16, 'p10': 31, 'p20': 63}

In [8]:
X_ = df1.drop(['class'], axis=1)
len(X_.columns)

312

In [9]:
paramXGBoost_grid1 = {"subsample":[0.5, 0.75, 1],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[5, 6, 7],
              "min_child_weight":[1,5,15],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [10]:
paramXGBoost_grid2 = {"subsample":[0.2,0.6,0.9],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[7,15,25],
              "min_child_weight":[0.1,0.5,1],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [11]:
paramXGBoost_grid3 = {"subsample":[0.2,0.6,0.9],
              "colsample_bytree":[0.1,0.25, 0.5],
              "max_depth":[10,15,20],
              "min_child_weight":[20,40,60],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [12]:
paramXGBoost_grid4 = {"subsample":[0.75,0.9,1],
              "colsample_bytree":[0.1,0.25, 0.5],
              "max_depth":[5,10,15],
              "min_child_weight":[5,10,20],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [13]:
param_d = {'set1':paramXGBoost_grid1,
           'set2':paramXGBoost_grid2,
           'set3':paramXGBoost_grid3,
           'set4':paramXGBoost_grid4
          }

param_d

{'set1': {'subsample': [0.5, 0.75, 1],
  'colsample_bytree': [0.5, 0.75, 1],
  'max_depth': [5, 6, 7],
  'min_child_weight': [1, 5, 15],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set2': {'subsample': [0.2, 0.6, 0.9],
  'colsample_bytree': [0.5, 0.75, 1],
  'max_depth': [7, 15, 25],
  'min_child_weight': [0.1, 0.5, 1],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set3': {'subsample': [0.2, 0.6, 0.9],
  'colsample_bytree': [0.1, 0.25, 0.5],
  'max_depth': [10, 15, 20],
  'min_child_weight': [20, 40, 60],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set4': {'subsample': [0.75, 0.9, 1],
  'colsample_bytree': [0.1, 0.25, 0.5],
  'max_depth': [5, 10, 15],
  'min_child_weight': [5, 10, 20],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]}}

# XGBoost

In [14]:
# baseline model

X = df1.drop(['class'], axis=1)
y = df1['class']
 


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123) 

""" do not need to split the data into training and test sets when using 
cross_val_score from the scikit-learn library. The purpose of 
cross_val_score is to perform cross-validation on the entire dataset, 
which automatically splits the data into training and validation sets 
for each fold. """
# XGBoost
xgbc = XGBClassifier(random_state=123)

## results F1 and stdF1 ##
avgF1_baselineXGBoost = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1_baselineXGBoost = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1_baselineXGBoost)
print('std F1 Score: ', stdF1_baselineXGBoost)



Mean F1 Score:  0.8643578643578642
std F1 Score:  0.04772011171804006


In [15]:
n_baselineXGBoost = len(X_.columns)
n_baselineXGBoost

312

In [16]:
# tuned baseline model_ parameterset1 


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid1,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)

print(grid_search.best_params_)


xgbc_tuned1 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned1.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set1_baselineXGBoost = cross_val_score(xgbc_tuned1, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set1_baselineXGBoost = cross_val_score(xgbc_tuned1, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set1_baselineXGBoost)
print('std F1 Score: ', stdF1set1_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 1, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.75}


Mean F1 Score:  0.9003102453102454
std F1 Score:  0.05479680704989319


In [17]:
# tuned baseline model_ parameterset2


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid2,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned2 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned2.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set2_baselineXGBoost = cross_val_score(xgbc_tuned2, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set2_baselineXGBoost = cross_val_score(xgbc_tuned2, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set2_baselineXGBoost)
print('std F1 Score: ', stdF1set2_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.5, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 0.5, 'subsample': 0.6}


Mean F1 Score:  0.8961062303709362
std F1 Score:  0.06205463659536259


In [18]:
# tuned baseline model_ parameterset3


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid3,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned3 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned3.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set3_baselineXGBoost = cross_val_score(xgbc_tuned3, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set3_baselineXGBoost = cross_val_score(xgbc_tuned3, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set3_baselineXGBoost)
print('std F1 Score: ', stdF1set3_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.1, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 20, 'subsample': 0.9}


Mean F1 Score:  0.7996536796536796
std F1 Score:  0.02055142088233746


In [19]:
# tuned baseline model_ parameterset4

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid4,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned4 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned4.fit(X,y)

## results F1 and stdF1 ##
avgF1set4_baselineXGBoost = cross_val_score(xgbc_tuned4, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set4_baselineXGBoost = cross_val_score(xgbc_tuned4, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set4_baselineXGBoost)
print('std F1 Score: ', stdF1set4_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.1, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 1}
Mean F1 Score:  0.9017390289449114
std F1 Score:  0.04974087795519576


In [20]:
output_basedlineXGBoost = {'n_baselineXGBoost':n_baselineXGBoost ,
                   'avgF1_baselineXGBoost':avgF1_baselineXGBoost ,
                   'stdF1_baselineXGBoost': stdF1_baselineXGBoost,
                   'avgF1set1_baselineXGBoost':avgF1set1_baselineXGBoost  ,
                   'stdF1set1_baselineXGBoost': stdF1set1_baselineXGBoost,
                   'avgF1set2_baselineXGBoost':avgF1set2_baselineXGBoost ,
                   'stdF1set2_baselineXGBoost':stdF1set2_baselineXGBoost ,
                   'avgF1set3_baselineXGBoost':avgF1set3_baselineXGBoost ,
                   'stdF1set3_baselineXGBoost':stdF1set3_baselineXGBoost ,
                   'avgF1set4_baselineXGBoost':avgF1set4_baselineXGBoost ,
                   'stdF1set4_baselineXGBoost':stdF1set4_baselineXGBoost ,
                     }
output_basedlineXGBoost

{'n_baselineXGBoost': 312,
 'avgF1_baselineXGBoost': 0.8643578643578642,
 'stdF1_baselineXGBoost': 0.04772011171804006,
 'avgF1set1_baselineXGBoost': 0.9003102453102454,
 'stdF1set1_baselineXGBoost': 0.05479680704989319,
 'avgF1set2_baselineXGBoost': 0.8961062303709362,
 'stdF1set2_baselineXGBoost': 0.06205463659536259,
 'avgF1set3_baselineXGBoost': 0.7996536796536796,
 'stdF1set3_baselineXGBoost': 0.02055142088233746,
 'avgF1set4_baselineXGBoost': 0.9017390289449114,
 'stdF1set4_baselineXGBoost': 0.04974087795519576}

In [21]:
output_basedlineXGBoost = pd.DataFrame([output_basedlineXGBoost])
output_basedlineXGBoost = output_basedlineXGBoost.transpose()
output_basedlineXGBoost.to_csv('output_basedlineXGBoost.csv', index=True)

## Feature Selection

# Filter

In [22]:
# Mutual Information: Filter method

mi_XGBoostscore = MIC(X,y)
mi_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_MI':mi_XGBoostscore})
mi_XGBoostfeatures['value_MI'] = mi_XGBoostfeatures['value_MI'].map('{:,.19f}'.format)
""" the more Mutual Information is the more feature importance it will have"""
mi_XGBoostfeatures = mi_XGBoostfeatures.sort_values(by='value_MI',ascending=False)
# mi_XGBoostfeatures = mi_XGBoostfeatures.set_index('Feature')
mi_XGBoostfeatures = mi_XGBoostfeatures.reset_index()
mi_XGBoostfeatures.to_csv('mi_XGBoostfeatures.csv', index=True)

mi_XGBoostfeatures

,index,Feature,value_MI
0,0,Jitter->F0_abs_dif,0.2471220337085198171
1,9,Jitter->F0_TKEO_prc5,0.2159629565098533099
2,10,Jitter->F0_TKEO_prc25,0.2136084295259312871
3,143,entropy_shannon_5_coef,0.2070833897803643353
4,152,entropy_log_4_coef,0.2011992950764724952
...,...,...,...
307,99,2nd delta,0.0000000000000000000
308,247,entropy_log3_8_coef,0.0000000000000000000
309,245,entropy_log3_6_coef,0.0000000000000000000
310,107,10th delta,0.0000000000000000000


In [23]:
# variance treshold: Filter method
"""Variance threshold is a simple method for feature selection that removes all features 
whose variance doesn't meet a certain threshold."""

threshold = 0.1
selector = VarianceThreshold(threshold)
selector = selector.fit(X)
variances = selector.variances_

variancethres_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_variancethres':variances})
# variancethres_XGBoostfeatures['value_variancethres'] = variancethres_XGBoostfeatures['value_variancethres'].map('{:,.19f}'.format)


"""The idea behind this method is that features 
with low variance don't contain much information and can be removed without affecting 
the performance of the model."""
variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.sort_values(by='value_variancethres',ascending=False)
# variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.set_index('Feature')
variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.reset_index()
variancethres_XGBoostfeatures.to_csv('variancethres_XGBoostfeatures.csv', index=True)

variancethres_XGBoostfeatures

,index,Feature,value_variancethres
0,219,Ea2,1.000000
1,110,delta delta log energy,1.000000
2,161,det_TKEO_mean_3_coef,1.000000
3,72,VFER->SNR_SEO,1.000000
4,259,det_TKEO_mean3_10_coef,1.000000
...,...,...,...
307,163,det_TKEO_mean_5_coef,1.000000
308,169,det_TKEO_std_1_coef,1.000000
309,226,Ed2_7_coef,1.000000
310,127,Data_length,1.000000


In [24]:
# MultiSURF: Filter method

features = list(X_.columns)

msrf = MultiSURFSelector(n_features=len(df1.columns)-1)
selected_cols = msrf.select(df1,'class')

list_of_numbers = list(range(1, len(df1.columns)))

MultiSURF_XGBoostfeatures = pd.DataFrame({'Feature':selected_cols,
                            'value_MultiSURF':list_of_numbers})
# # MultiSURF_XGBoostfeatures['value_MultiSURF'] = MultiSURF_XGBoostfeatures['value_MultiSURF'].map('{:,.19f}'.format)
# เรียงจากเลขน้อยคือสำคัญสูงสุด
MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.sort_values(by='value_MultiSURF',ascending=True)
# MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.set_index('Feature')
MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.reset_index()

MultiSURF_XGBoostfeatures.to_csv('MultiSURF_XGBoostfeatures.csv', index=True)

MultiSURF_XGBoostfeatures

,index,Feature,value_MultiSURF
0,0,Data_length,1
1,1,entropy_shannon_8_coef,2
2,2,entropy_shannon3_8_coef,3
3,3,det_TKEO_mean_7_coef,4
4,4,entropy_shannon_7_coef,5
...,...,...,...
307,307,10th delta,308
308,308,MFCC_8th coef,309
309,309,5th delta,310
310,310,4th delta,311


# Wrapper

In [25]:
# SHAP: Wrapper method


# Explain model predictions using shap library:
explainer = shap.TreeExplainer(xgbc_tuned4) #### 
shap_values = explainer.shap_values(X)
vals= np.abs(shap_values).mean(0)

SHAP_XGBoostfeatures= pd.DataFrame({'Feature':list(X_.columns),
                                 'value_SHAP':vals})
SHAP_XGBoostfeatures['value_SHAP'] = SHAP_XGBoostfeatures['value_SHAP'].map('{:,.19f}'.format)
SHAP_XGBoostfeatures = SHAP_XGBoostfeatures.sort_values(by=['value_SHAP'],ascending=False)
SHAP_XGBoostfeatures = SHAP_XGBoostfeatures.set_index('Feature')
SHAP_XGBoostfeatures.to_csv('SHAP_XGBoostfeatures.csv', index=True)

SHAP_XGBoostfeatures

,value_SHAP
Feature,
MFCC_2nd coef,0.3918479681015014648
GNE->NSR_SEO,0.3068278729915618896
Log energy,0.2385229170322418213
F0_series_F0_expTitze,0.2027625888586044312
Jitter->pitch_PQ5_classical_Schoentgen,0.1940937191247940063
...,...
Ed_3_coef,0.0000000000000000000
Ed_4_coef,0.0000000000000000000
Ed_5_coef,0.0000000000000000000


In [26]:
# Recursive Feature Elimination(RFE): Wrapper method


"""In this example, the parameter "n_features_to_select" is set to 1, 
which means that only one feature will be selected at each iteration 
of RFE"""

rfe = RFE(estimator=xgbc_tuned4, n_features_to_select=1) #### changehere
rfe.fit(X,y)

rfe_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_rfe':rfe.ranking_})

# rfe_XGBoostfeatures['value_XGBooste'] = rfe_XGBoostfeatures['value_XGBooste'].astype(int)
# rfe_XGBoostfeatures['value_XGBooste'] = rfe_XGBoostfeatures['value_XGBooste'].map('{:,.19f}'.format)
"""The RFE assigns a ranking value to each feature, typically starting from 1 for 
the most important feature, and increasing for the less important features."""
rfe_XGBoostfeatures = rfe_XGBoostfeatures.sort_values(by=['value_rfe'],ascending=True)
rfe_XGBoostfeatures = rfe_XGBoostfeatures.set_index('Feature')

rfe_XGBoostfeatures.to_csv('rfe_XGBoostfeatures.csv', index=True)

rfe_XGBoostfeatures

,value_rfe
Feature,
entropy_log_4_coef,1
MFCC_2nd coef,2
Jitter->pitch_PQ5_classical_Schoentgen,3
entropy_shannon_5_coef,4
prc95_5_F0_series_F0_expTitze,5
...,...
entropy_log_5_coef,308
entropy_log_7_coef,309
entropy_log_6_coef,310


In [27]:
# Boruta: Wrapper method




feat_selector = BorutaPy(xgbc_tuned4, n_estimators='auto', verbose=2, random_state=1) #### changehere
# feat_selector.fit(X_train, y_train)
feat_selector = feat_selector.fit(np.array(X), np.array(y)) 
features = X.columns
feature_importance = feat_selector.ranking_
# feature_importance
boruta_XGBoostfeatures = pd.DataFrame(list(zip(X_.columns, feature_importance[0:].tolist())), 
                                  columns=['Feature','value_boruta'])

# output the feature importance ranking: 1 is most importance >ascending=True
boruta_XGBoostfeatures = boruta_XGBoostfeatures.sort_values(by=['value_boruta'],ascending=True)
boruta_XGBoostfeatures = boruta_XGBoostfeatures.set_index('Feature')

boruta_XGBoostfeatures.to_csv('boruta_XGBoostfeatures.csv', index=True)

boruta_XGBoostfeatures

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	312
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	62
Rejected: 	250
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	62
Rejected: 	250
Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	62
Rejected: 	250
Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	62
Rejected: 	250
Iteration: 	12 / 100
Confirmed: 	0
Tentative: 	53
Rejected: 	259
Iteration: 	13 / 100
Confirmed: 	0
Tentative: 	53
Rejected: 	259
Iteration: 	14 / 100
Confirmed: 	0
Tentative: 	53
Rejected: 	259
Iteration: 	15 / 100
Confirmed: 	0
Tentative: 	53
Rejected: 	259
Iteration: 	16 / 100
Confirmed: 	0
Tentat

,value_boruta
Feature,
Jitter->F0_abs_dif,1
entropy_log_4_coef,1
std_F0_series_F0_expTitze,1
det_TKEO_mean_8_coef,1
DFA,1
...,...
entropy_shannon_4_coef,189
entropy_shannon_6_coef,189
entropy_log_1_coef,189


# single FS

# Filter

In [28]:
method_Filter = {'MI': mi_XGBoostfeatures,
          'variancethres': variancethres_XGBoostfeatures,
          'MultiSURF': MultiSURF_XGBoostfeatures
         }

In [29]:
output_filter = dict()


for tres,t in thres_d.items():
    for met,m in method_Filter.items():
        k =(filter_arr(m.filter(regex='value').squeeze(), t-1))   
        if k[1] != 0:
            n = k[1]+1 
            print("n"+"_single"+met+"_"+tres+"XGBoost",n)
            output_filter["n"+"_single"+met+"_"+tres+"XGBoost"] = n
            df_selected = m.iloc[:k[1]+1]
            df_filter = df1.filter(items=df_selected['Feature'])
            X = df_filter
            y = df1['class']
            
            ## before tuned ##
            xgbc = XGBClassifier(random_state=123)
            xgbc.fit(X,y)
            ## results F1 and stdF1 ##
            m = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() 
            s = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() 
            print("avgF1"+"_single"+met+"_"+tres+"XGBoost" , m)
            print("stdF1"+"_single"+met+"_"+tres+"XGBoost",s)
            output_filter["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = m
            output_filter["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = s
            
            for parm, p in param_d.items():
                # Instantiate the grid search model
                grid_search = GridSearchCV(estimator = xgbc, param_grid = p,
                                        cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
                # Fit the grid search to the data
                grid_search.fit(X,y)
        #         print(grid_search.best_params_)
                xgbc_tuned = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                                max_depth=grid_search.best_params_['max_depth'],
                                                colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                                min_child_weight=grid_search.best_params_['min_child_weight'],
                                                learning_rate=grid_search.best_params_['learning_rate'],
                                                eta=grid_search.best_params_['eta'])
                xgbc_tuned.fit(X,y)

                ## results F1 and stdF1 ##
                m_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).mean() 
                s_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).std() 
                print("avgF1"+parm+"_single"+met+"_"+tres+"XGBoost" , m_tune)
                print("stdF1"+parm+"_single"+met+"_"+tres+"XGBoost",s_tune)
                output_filter["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = m_tune
                output_filter["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = s_tune

        else:
            output_filter["n"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_filter["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_filter["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            for parm, p in param_d.items():
                output_filter["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
                output_filter["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
         

        


n_singleMI_base2nXGBoost 7
avgF1_singleMI_base2nXGBoost 0.7720696443219662
stdF1_singleMI_base2nXGBoost 0.13859793622453048
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set1_singleMI_base2nXGBoost 0.8276521053765634
stdF1set1_singleMI_base2nXGBoost 0.11723963541498901
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set2_singleMI_base2nXGBoost 0.8368250377073906
stdF1set2_singleMI_base2nXGBoost 0.11536066772061342
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set3_singleMI_base2nXGBoost 0.7996536796536796
stdF1set3_singleMI_base2nXGBoost 0.02055142088233746
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set4_singleMI_base2nXGBoost 0.8489923337756153
stdF1set4_singleMI_base2nXGBoost 0.1287265356043338
n_singlevariancethres_base2nXGBoost 3
avgF1_singlevariancethres_base2nXGBoost 0.8139094058676102
stdF1_singlevariancethres_base2nXGBoost 0.07767029649821591
Fitting 5 folds for each of 729 candidates, 

In [30]:
output_filter

{'n_singleMI_base2nXGBoost': 7,
 'avgF1_singleMI_base2nXGBoost': 0.7720696443219662,
 'stdF1_singleMI_base2nXGBoost': 0.13859793622453048,
 'avgF1set1_singleMI_base2nXGBoost': 0.8276521053765634,
 'stdF1set1_singleMI_base2nXGBoost': 0.11723963541498901,
 'avgF1set2_singleMI_base2nXGBoost': 0.8368250377073906,
 'stdF1set2_singleMI_base2nXGBoost': 0.11536066772061342,
 'avgF1set3_singleMI_base2nXGBoost': 0.7996536796536796,
 'stdF1set3_singleMI_base2nXGBoost': 0.02055142088233746,
 'avgF1set4_singleMI_base2nXGBoost': 0.8489923337756153,
 'stdF1set4_singleMI_base2nXGBoost': 0.1287265356043338,
 'n_singlevariancethres_base2nXGBoost': 3,
 'avgF1_singlevariancethres_base2nXGBoost': 0.8139094058676102,
 'stdF1_singlevariancethres_base2nXGBoost': 0.07767029649821591,
 'avgF1set1_singlevariancethres_base2nXGBoost': 0.8442378338895367,
 'stdF1set1_singlevariancethres_base2nXGBoost': 0.046392444546284016,
 'avgF1set2_singlevariancethres_base2nXGBoost': 0.8379108645052918,
 'stdF1set2_singlevarian

In [31]:

output_single_Filter_XGBoost = pd.DataFrame([output_filter])
output_single_Filter_XGBoost = output_single_Filter_XGBoost.transpose()
output_single_Filter_XGBoost.to_csv('output_single_Filter_XGBoost.csv', index=True)

# Wrapper

In [32]:
SHAP_XGBoostfeatures = pd.read_csv('SHAP_XGBoostfeatures.csv')
rfe_XGBoostfeatures = pd.read_csv('rfe_XGBoostfeatures.csv')
boruta_XGBoostfeatures = pd.read_csv('boruta_XGBoostfeatures.csv')

In [33]:
method_Wrapper = {  'SHAP':SHAP_XGBoostfeatures,
                    'rfe': rfe_XGBoostfeatures,
                    'boruta': boruta_XGBoostfeatures
                }

In [34]:
output_wrapper = dict()


for tres,t in thres_d.items():
    for met,m in method_Wrapper.items():
        k =(filter_arr(m.filter(regex='value').squeeze(), t-1))   
        if k[1] != 0:
            n = k[1]+1 
            print("n"+"_single"+met+"_"+tres+"XGBoost",n)
            output_wrapper["n"+"_single"+met+"_"+tres+"XGBoost"] = n
            df_selected = m.iloc[:k[1]+1]
            df_filter = df1.filter(items=df_selected['Feature'])
            X = df_filter
            y = df1['class']
            
            ## before tuned ##
            xgbc = XGBClassifier(random_state=123)
            xgbc.fit(X,y)
            ## results F1 and stdF1 ##
            m = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() 
            s = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() 
            print("avgF1"+"_single"+met+"_"+tres+"XGBoost" , m)
            print("stdF1"+"_single"+met+"_"+tres+"XGBoost",s)
            output_wrapper["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = m
            output_wrapper["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = s
            
            for parm, p in param_d.items():
                # Instantiate the grid search model
                grid_search = GridSearchCV(estimator = xgbc, param_grid = p,
                                        cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
                # Fit the grid search to the data
                grid_search.fit(X,y)
        #         print(grid_search.best_params_)
                xgbc_tuned = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                                max_depth=grid_search.best_params_['max_depth'],
                                                colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                                min_child_weight=grid_search.best_params_['min_child_weight'],
                                                learning_rate=grid_search.best_params_['learning_rate'],
                                                eta=grid_search.best_params_['eta'])
                xgbc_tuned.fit(X,y)

                ## results F1 and stdF1 ##
                m_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).mean() 
                s_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).std() 
                print("avgF1"+parm+"_single"+met+"_"+tres+"XGBoost" , m_tune)
                print("stdF1"+parm+"_single"+met+"_"+tres+"XGBoost",s_tune)
                output_wrapper["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = m_tune
                output_wrapper["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = s_tune

        else:
            output_wrapper["n"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_wrapper["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_wrapper["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            for parm, p in param_d.items():
                output_wrapper["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
                output_wrapper["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
         

        


n_singleSHAP_base2nXGBoost 7
avgF1_singleSHAP_base2nXGBoost 0.9203479286451423
stdF1_singleSHAP_base2nXGBoost 0.044146955569844305
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set1_singleSHAP_base2nXGBoost 0.9003186888790605
stdF1set1_singleSHAP_base2nXGBoost 0.05253382766289704
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set2_singleSHAP_base2nXGBoost 0.9119007076514816
stdF1set2_singleSHAP_base2nXGBoost 0.04303135105353047
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set3_singleSHAP_base2nXGBoost 0.7996536796536796
stdF1set3_singleSHAP_base2nXGBoost 0.02055142088233746
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set4_singleSHAP_base2nXGBoost 0.9194707356626862
stdF1set4_singleSHAP_base2nXGBoost 0.027564712853517293
n_singlerfe_base2nXGBoost 7
avgF1_singlerfe_base2nXGBoost 0.8978527294858566
stdF1_singlerfe_base2nXGBoost 0.06221018551553827
Fitting 5 folds for each of 729 candidates, total

In [35]:
output_wrapper

{'n_singleSHAP_base2nXGBoost': 7,
 'avgF1_singleSHAP_base2nXGBoost': 0.9203479286451423,
 'stdF1_singleSHAP_base2nXGBoost': 0.044146955569844305,
 'avgF1set1_singleSHAP_base2nXGBoost': 0.9003186888790605,
 'stdF1set1_singleSHAP_base2nXGBoost': 0.05253382766289704,
 'avgF1set2_singleSHAP_base2nXGBoost': 0.9119007076514816,
 'stdF1set2_singleSHAP_base2nXGBoost': 0.04303135105353047,
 'avgF1set3_singleSHAP_base2nXGBoost': 0.7996536796536796,
 'stdF1set3_singleSHAP_base2nXGBoost': 0.02055142088233746,
 'avgF1set4_singleSHAP_base2nXGBoost': 0.9194707356626862,
 'stdF1set4_singleSHAP_base2nXGBoost': 0.027564712853517293,
 'n_singlerfe_base2nXGBoost': 7,
 'avgF1_singlerfe_base2nXGBoost': 0.8978527294858566,
 'stdF1_singlerfe_base2nXGBoost': 0.06221018551553827,
 'avgF1set1_singlerfe_base2nXGBoost': 0.8978527294858566,
 'stdF1set1_singlerfe_base2nXGBoost': 0.06221018551553827,
 'avgF1set2_singlerfe_base2nXGBoost': 0.904704776647501,
 'stdF1set2_singlerfe_base2nXGBoost': 0.05444276242063975,
 '

In [36]:

output_single_wrapper_XGBoost = pd.DataFrame([output_wrapper])
output_single_wrapper_XGBoost = output_single_wrapper_XGBoost.transpose()
output_single_wrapper_XGBoost.to_csv('output_single_wrapper_XGBoost.csv', index=True)